In [1]:
import pandas as pd
import numpy as np
import operator
import seaborn as sns
import matplotlib.pyplot as plt
import visuals as vs

from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler
from time import time
from IPython.display import display
from io import StringIO
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score,accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score, fbeta_score


In [2]:
train = pd.read_csv('wr_data.csv')
train.head(30)

,Unnamed: 0,Player,avoided_tackles,grades_offense,longest,receptions,routes,targets,yards,yards_after_catch,yprr,ypt,"1,000 Yard Seasons"
0,0,Drake London,8.0,83.2,40.0,72.0,419.0,114.0,866.0,231.0,2.07,7.596491,0
1,1,Garrett Wilson,22.0,82.7,60.0,83.0,595.0,139.0,1103.0,389.0,1.85,7.935252,2
2,2,Chris Olave,2.0,82.5,53.0,72.0,431.0,114.0,1044.0,219.0,2.42,9.157895,2
3,3,Jameson Williams,0.0,63.4,41.0,1.0,37.0,8.0,41.0,5.0,1.11,5.125000,0
4,4,Jahan Dotson,2.0,70.5,61.0,35.0,376.0,56.0,523.0,144.0,1.39,9.339286,0
5,5,Treylon Burks,4.0,74.1,51.0,33.0,254.0,53.0,444.0,177.0,1.75,8.377358,0
6,6,Christian Watson,4.0,77.1,63.0,41.0,270.0,65.0,611.0,264.0,2.26,9.400000,0
7,7,Wan'Dale Robinson,3.0,72.0,19.0,23.0,129.0,30.0,227.0,127.0,1.76,7.566667,0
8,8,Tyquan Thornton,1.0,55.2,29.0,22.0,326.0,43.0,247.0,66.0,0.76,5.744186,0
9,9,George Pickens,7.0,68.8,42.0,52.0,580.0,84.0,801.0,110.0,1.38,9.535714,1


In [3]:
test = pd.read_csv('rookie_23.csv')
test.head(1)


,Unnamed: 0,Player,avoided_tackles,grades_offense,longest,receptions,routes,targets,yards,yards_after_catch,yprr,ypt
0,0,Jaxon Smith-Njigba,6.0,63.3,35.0,63.0,477.0,90.0,628.0,370.0,1.32,6.977778


In [4]:


def onek_fun(x):
    if x == 0:
        x = 0
    else:
        x = 1
    return x


In [9]:
train['1,000 Yard Seasons'] = train['1,000 Yard Seasons'].apply(onek_fun)
train.head(30)

,Unnamed: 0,Player,avoided_tackles,grades_offense,longest,receptions,routes,targets,yards,yards_after_catch,yprr,ypt,"1,000 Yard Seasons"
0,0,Drake London,8.0,83.2,40.0,72.0,419.0,114.0,866.0,231.0,2.07,7.596491,0
1,1,Garrett Wilson,22.0,82.7,60.0,83.0,595.0,139.0,1103.0,389.0,1.85,7.935252,1
2,2,Chris Olave,2.0,82.5,53.0,72.0,431.0,114.0,1044.0,219.0,2.42,9.157895,1
3,3,Jameson Williams,0.0,63.4,41.0,1.0,37.0,8.0,41.0,5.0,1.11,5.125000,0
4,4,Jahan Dotson,2.0,70.5,61.0,35.0,376.0,56.0,523.0,144.0,1.39,9.339286,0
5,5,Treylon Burks,4.0,74.1,51.0,33.0,254.0,53.0,444.0,177.0,1.75,8.377358,0
6,6,Christian Watson,4.0,77.1,63.0,41.0,270.0,65.0,611.0,264.0,2.26,9.400000,0
7,7,Wan'Dale Robinson,3.0,72.0,19.0,23.0,129.0,30.0,227.0,127.0,1.76,7.566667,0
8,8,Tyquan Thornton,1.0,55.2,29.0,22.0,326.0,43.0,247.0,66.0,0.76,5.744186,0
9,9,George Pickens,7.0,68.8,42.0,52.0,580.0,84.0,801.0,110.0,1.38,9.535714,1


In [12]:
test2 = test
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          346 non-null    int64  
 1   Player              346 non-null    object 
 2   avoided_tackles     346 non-null    float64
 3   grades_offense      346 non-null    float64
 4   longest             346 non-null    float64
 5   receptions          346 non-null    float64
 6   routes              346 non-null    float64
 7   targets             346 non-null    float64
 8   yards               346 non-null    float64
 9   yards_after_catch   346 non-null    float64
 10  yprr                346 non-null    float64
 11  ypt                 346 non-null    float64
 12  1,000 Yard Seasons  346 non-null    int64  
dtypes: float64(10), int64(2), object(1)
memory usage: 35.3+ KB


In [14]:
features = [#'avoided_tackles', 'grades_offense', 'longest',
       'receptions', 
    #'routes',
    'targets', 'yards', 
    #'yards_after_catch',
    'yprr',
       'ypt']
prediction_class = ['1,000 Yard Seasons']

In [16]:
X = train[features].values

In [18]:
Y = train[prediction_class].values

In [20]:
 type(X), type(Y)


(numpy.ndarray, numpy.ndarray)

In [22]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [24]:


clf = GradientBoostingClassifier(n_estimators=100,  random_state=1)

clf.fit(x_train[:310], y_train[:310].ravel())
print("Training set has {} samples.".format(x_train.shape[0]))
print("Testing set has {} samples.".format(x_test.shape[0]))

Training set has 276 samples.
Testing set has 70 samples.


In [26]:
predictions = clf.predict(x_test)
predictions


array([1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int64)

In [28]:

mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.12857142857142856


In [31]:
print('The accuracy is: ', accuracy_score(y_test, predictions)*100)

The accuracy is:  87.14285714285714


In [33]:
fscore = clf.score(x_test, y_test)
print(fscore)

0.8714285714285714


In [35]:
results = pd.DataFrame()
results[['Actual Values']] = y_test
results['Predictions'] = predictions
print(results)

    Actual Values  Predictions
0               1            1
1               0            0
2               1            1
3               0            0
4               0            0
..            ...          ...
65              0            0
66              0            0
67              0            0
68              0            0
69              0            0

[70 rows x 2 columns]


In [37]:
# Initialize the classifier
gb = GradientBoostingClassifier(random_state = 1)



# parameters
parameters = {'n_estimators' :[50, 100, 150, 200, 250], 'learning_rate': [0.01, 0.05, .1, .5, 1.0], 'max_depth': [3, 5, 7, 8, 9]}



grid_search = GridSearchCV(gb, parameters, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train.ravel())

best_params = grid_search.best_params_




In [38]:
best_params


{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50}

In [45]:
clf2 = GradientBoostingClassifier(n_estimators=50, learning_rate=0.05, max_depth=3, random_state=1)

clf2.fit(x_train[:310], y_train[:310].ravel())
predictions = clf2.predict(x_test)
mse = mean_squared_error(y_test, predictions)
fscore = clf2.score(x_test, y_test)
print("Mean Squared Error:", mse)
print('The accuracy is: ', accuracy_score(y_test, predictions)*100)
print(fscore)

Mean Squared Error: 0.11428571428571428
The accuracy is:  88.57142857142857
0.8857142857142857


In [47]:
results = pd.DataFrame()
results[['Actual Values']] = y_test
results['Predictions'] = predictions
print(results)

    Actual Values  Predictions
0               1            1
1               0            0
2               1            1
3               0            0
4               0            0
..            ...          ...
65              0            0
66              0            0
67              0            0
68              0            0
69              0            0

[70 rows x 2 columns]


In [53]:
test_23 = pd.read_csv('rookie_23.csv')
test_23.head(1)

,Unnamed: 0,Player,avoided_tackles,grades_offense,longest,receptions,routes,targets,yards,yards_after_catch,yprr,ypt
0,0,Jaxon Smith-Njigba,6.0,63.3,35.0,63.0,477.0,90.0,628.0,370.0,1.32,6.977778


In [55]:
import joblib
#saving ML model

joblib.dump(clf2, "model.pkl") 

['model.pkl']

In [62]:
clf3 = joblib.load("model.pkl")
X = test_23[[ #'avoided_tackles', 'grades_offense', 'longest',
       'receptions', 
    #'routes',
    'targets', 'yards', 
    #'yards_after_catch',
    'yprr',
       'ypt']]
predictions = clf3.predict(X)

C:\Users\remsteele\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [64]:
lista = predictions.tolist()

In [66]:
db1 = test_23['Player']

In [68]:
clf3_db = pd.DataFrame(lista, columns=['Future 1k Seasons'])

In [70]:
# 1 shows players it predicts to have future 1k yard seasons
df_result = pd.concat([db1, clf3_db],axis=1, ignore_index=True, sort=False)
df_result

,0,1
0,Jaxon Smith-Njigba,0
1,Quentin Johnston,0
2,Zay Flowers,1
3,Jordan Addison,1
4,Jonathan Mingo,0
5,Jayden Reed,1
6,Rashee Rice,1
7,Tank Dell,1
8,Jalin Hyatt,0
9,Cedric Tillman,0
